In [1]:
from os.path import join as pjoin

import os
import numpy as np
import torch
from tqdm import tqdm

from common.skeleton import Skeleton
from common.quaternion import *
from paramUtil import *

# ------------------ helper functions ------------------ #

def uniform_skeleton(positions, target_offset):
    src_skel = Skeleton(n_raw_offsets, kinematic_chain, 'cpu')
    src_offset = src_skel.get_offsets_joints(torch.from_numpy(positions[0]))
    src_offset = src_offset.numpy()
    tgt_offset = target_offset.numpy()

    # 다리 길이 비율로 스케일 맞추기
    src_leg_len = np.abs(src_offset[l_idx1]).max() + np.abs(src_offset[l_idx2]).max()
    tgt_leg_len = np.abs(tgt_offset[l_idx1]).max() + np.abs(tgt_offset[l_idx2]).max()
    scale_rt = tgt_leg_len / src_leg_len

    src_root_pos = positions[:, 0]
    tgt_root_pos = src_root_pos * scale_rt

    # IK -> 포즈를 quaternion으로
    quat_params = src_skel.inverse_kinematics_np(positions, face_joint_indx)

    # FK -> 타겟 스켈레톤으로 forward kinematics
    src_skel.set_offset(target_offset)
    new_joints = src_skel.forward_kinematics_np(quat_params, tgt_root_pos)
    return new_joints


def process_file(positions, feet_thre):
    # positions: (seq_len, joints_num, 3)

    # 1) 스켈레톤 통일
    positions = uniform_skeleton(positions, tgt_offsets)

    # 2) 바닥에 붙이기 (y축 최소를 0으로)
    floor_height = positions.min(axis=0).min(axis=0)[1]
    positions[:, :, 1] -= floor_height

    # 3) XZ 원점 정렬
    root_pos_init = positions[0]
    root_pose_init_xz = root_pos_init[0] * np.array([1, 0, 1])
    positions = positions - root_pose_init_xz

    # 4) 초기 방향을 Z+ 로 맞추기
    r_hip, l_hip, sdr_r, sdr_l = face_joint_indx
    across1 = root_pos_init[r_hip] - root_pos_init[l_hip]
    across2 = root_pos_init[sdr_r] - root_pos_init[sdr_l]
    across = across1 + across2
    across = across / np.sqrt((across ** 2).sum(axis=-1))[..., np.newaxis]

    forward_init = np.cross(np.array([[0, 1, 0]]), across, axis=-1)
    forward_init = forward_init / np.sqrt((forward_init ** 2).sum(axis=-1))[..., np.newaxis]

    target = np.array([[0, 0, 1]])
    root_quat_init = qbetween_np(forward_init, target)
    root_quat_init = np.ones(positions.shape[:-1] + (4,)) * root_quat_init

    positions_b = positions.copy()
    positions = qrot_np(root_quat_init, positions)

    global_positions = positions.copy()

    # 5) 발 접촉 탐지
    def foot_detect(positions, thres):
        velfactor = np.array([thres, thres])

        feet_l_x = (positions[1:, fid_l, 0] - positions[:-1, fid_l, 0]) ** 2
        feet_l_y = (positions[1:, fid_l, 1] - positions[:-1, fid_l, 1]) ** 2
        feet_l_z = (positions[1:, fid_l, 2] - positions[:-1, fid_l, 2]) ** 2
        feet_l = ((feet_l_x + feet_l_y + feet_l_z) < velfactor).astype(np.float32)

        feet_r_x = (positions[1:, fid_r, 0] - positions[:-1, fid_r, 0]) ** 2
        feet_r_y = (positions[1:, fid_r, 1] - positions[:-1, fid_r, 1]) ** 2
        feet_r_z = (positions[1:, fid_r, 2] - positions[:-1, fid_r, 2]) ** 2
        feet_r = ((feet_r_x + feet_r_y + feet_r_z) < velfactor).astype(np.float32)
        return feet_l, feet_r

    feet_l, feet_r = foot_detect(positions, feet_thre)

    r_rot = None

    def get_rifke(positions):
        # root 좌표 기준 local로
        positions[..., 0] -= positions[:, 0:1, 0]
        positions[..., 2] -= positions[:, 0:1, 2]
        # 모든 포즈를 Z+로 맞추기
        positions_ = qrot_np(np.repeat(r_rot[:, None], positions.shape[1], axis=1), positions)
        return positions_

    def get_cont6d_params(positions):
        skel = Skeleton(n_raw_offsets, kinematic_chain, "cpu")
        quat_params = skel.inverse_kinematics_np(positions, face_joint_indx, smooth_forward=True)

        cont_6d_params = quaternion_to_cont6d_np(quat_params)
        r_rot_local = quat_params[:, 0].copy()

        velocity = (positions[1:, 0] - positions[:-1, 0]).copy()
        velocity = qrot_np(r_rot_local[1:], velocity)

        r_velocity = qmul_np(r_rot_local[1:], qinv_np(r_rot_local[:-1]))
        return cont_6d_params, r_velocity, velocity, r_rot_local

    cont_6d_params, r_velocity, velocity, r_rot = get_cont6d_params(positions)
    positions = get_rifke(positions)

    # root height
    root_y = positions[:, 0, 1:2]

    # root rotation / linear velocity
    r_velocity = np.arcsin(r_velocity[:, 2:3])
    l_velocity = velocity[:, [0, 2]]
    root_data = np.concatenate([r_velocity, l_velocity, root_y[:-1]], axis=-1)

    # joint rotation (cont6d)
    rot_data = cont_6d_params[:, 1:].reshape(len(cont_6d_params), -1)

    # joint local position (ric)
    ric_data = positions[:, 1:].reshape(len(positions), -1)

    # local velocity
    local_vel = qrot_np(
        np.repeat(r_rot[:-1, None], global_positions.shape[1], axis=1),
        global_positions[1:] - global_positions[:-1]
    )
    local_vel = local_vel.reshape(len(local_vel), -1)

    data = root_data
    data = np.concatenate([data, ric_data[:-1]], axis=-1)
    data = np.concatenate([data, rot_data[:-1]], axis=-1)
    data = np.concatenate([data, local_vel], axis=-1)
    data = np.concatenate([data, feet_l, feet_r], axis=-1)

    return data, global_positions, positions, l_velocity


def recover_root_rot_pos(data):
    rot_vel = data[..., 0]
    r_rot_ang = torch.zeros_like(rot_vel).to(data.device)
    r_rot_ang[..., 1:] = rot_vel[..., :-1]
    r_rot_ang = torch.cumsum(r_rot_ang, dim=-1)

    r_rot_quat = torch.zeros(data.shape[:-1] + (4,)).to(data.device)
    r_rot_quat[..., 0] = torch.cos(r_rot_ang)
    r_rot_quat[..., 2] = torch.sin(r_rot_ang)

    r_pos = torch.zeros(data.shape[:-1] + (3,)).to(data.device)
    r_pos[..., 1:, [0, 2]] = data[..., :-1, 1:3]
    r_pos = qrot(qinv(r_rot_quat), r_pos)
    r_pos = torch.cumsum(r_pos, dim=-2)
    r_pos[..., 1] = data[..., 3]
    return r_rot_quat, r_pos


def recover_from_rot(data, joints_num, skeleton):
    r_rot_quat, r_pos = recover_root_rot_pos(data)
    r_rot_cont6d = quaternion_to_cont6d(r_rot_quat)

    start_indx = 1 + 2 + 1 + (joints_num - 1) * 3
    end_indx = start_indx + (joints_num - 1) * 6
    cont6d_params = data[..., start_indx:end_indx]

    cont6d_params = torch.cat([r_rot_cont6d, cont6d_params], dim=-1)
    cont6d_params = cont6d_params.view(-1, joints_num, 6)
    positions = skeleton.forward_kinematics_cont6d(cont6d_params, r_pos)
    return positions


def recover_from_ric(data, joints_num):
    r_rot_quat, r_pos = recover_root_rot_pos(data)
    positions = data[..., 4:(joints_num - 1) * 3 + 4]
    positions = positions.view(positions.shape[:-1] + (-1, 3))

    positions = qrot(
        qinv(r_rot_quat[..., None, :]).expand(positions.shape[:-1] + (4,)),
        positions
    )

    positions[..., 0] += r_pos[..., 0:1]
    positions[..., 2] += r_pos[..., 2:3]

    positions = torch.cat([r_pos.unsqueeze(-2), positions], dim=-2)
    return positions


# ------------------ main for HumanML3D ------------------ #

if __name__ == "__main__":
    # 예시로 사용할 클립 id (joints에 실제로 존재해야 함)
    example_id = "000021"

    # Lower legs indices
    l_idx1, l_idx2 = 5, 8
    # Right/Left foot joint indices
    fid_r, fid_l = [8, 11], [7, 10]
    # Face direction, r_hip, l_hip, sdr_r, sdr_l
    face_joint_indx = [2, 1, 17, 16]
    # hips
    r_hip, l_hip = 2, 1
    joints_num = 22

    data_dir = './joints/'
    save_dir1 = './HumanML3D/new_joints/'
    save_dir2 = './HumanML3D/new_joint_vecs/'

    os.makedirs(save_dir1, exist_ok=True)
    os.makedirs(save_dir2, exist_ok=True)

    # paramUtil 에서 가져온 raw offsets / kinematic chain
    n_raw_offsets = torch.from_numpy(t2m_raw_offsets)
    kinematic_chain = t2m_kinematic_chain

    # 타겟 스켈레톤 offset 추출
    example_path = os.path.join(data_dir, example_id + '.npy')
    if not os.path.exists(example_path):
        raise FileNotFoundError(f"Example file not found: {example_path}")

    example_data = np.load(example_path)
    example_data = example_data.reshape(len(example_data), -1, 3)
    example_data = torch.from_numpy(example_data)
    tgt_skel = Skeleton(n_raw_offsets, kinematic_chain, 'cpu')
    tgt_offsets = tgt_skel.get_offsets_joints(example_data[0])

    # joints 폴더 안에서 .npy만 처리 (Windows 안전)
    source_list = sorted(
        [f for f in os.listdir(data_dir) if f.lower().endswith('.npy')]
    )

    frame_num = 0
    for source_file in tqdm(source_list, desc="Processing joints"):
        src_path = os.path.join(data_dir, source_file)

        try:
            source_data = np.load(src_path)[:, :joints_num]
        except Exception as e:
            print(f"[SKIP] cannot load {src_path}: {e}")
            continue

        try:
            data, ground_positions, positions, l_velocity = process_file(source_data, 0.002)
            rec_ric_data = recover_from_ric(
                torch.from_numpy(data).unsqueeze(0).float(),
                joints_num
            )
            np.save(pjoin(save_dir1, source_file), rec_ric_data.squeeze().numpy())
            np.save(pjoin(save_dir2, source_file), data)
            frame_num += data.shape[0]
        except Exception as e:
            print(f"[ERROR] {source_file}: {e}")

    print('Total clips: %d, Frames: %d, Duration: %fm' %
          (len(source_list), frame_num, frame_num / 20 / 60))

    # 옵션: reference 파일 체크 (있을 때만)
    ref_joints_path = './HumanML3D/new_joints/012314.npy'
    ref_vecs_path = './HumanML3D/new_joint_vecs/012314.npy'
    if os.path.exists(ref_joints_path) and os.path.exists(ref_vecs_path):
        reference1 = np.load(ref_joints_path)
        reference2 = np.load(ref_vecs_path)
        print("Loaded reference clips 012314 for sanity check.")
    else:
        print("Reference files 012314 not found yet (this is normal on first run).")

Processing joints:  33%|███████████████████▎                                      | 9714/29232 [03:47<08:13, 39.55it/s]

[ERROR] 009707.npy: cannot reshape array of size 0 into shape (0,newaxis)


Processing joints:  38%|█████████████████████▌                                   | 11066/29232 [04:20<06:39, 45.48it/s]

[ERROR] 011059.npy: cannot reshape array of size 0 into shape (0,newaxis)


Processing joints:  83%|███████████████████████████████████████████████▍         | 24332/29232 [09:33<01:59, 40.86it/s]

[ERROR] M009707.npy: cannot reshape array of size 0 into shape (0,newaxis)


Processing joints:  88%|██████████████████████████████████████████████████       | 25683/29232 [10:05<01:18, 45.47it/s]

[ERROR] M011059.npy: cannot reshape array of size 0 into shape (0,newaxis)


Processing joints: 100%|█████████████████████████████████████████████████████████| 29232/29232 [11:28<00:00, 42.49it/s]

Total clips: 29232, Frames: 4117392, Duration: 3431.160000m
Loaded reference clips 012314 for sanity check.


### Check if your data is correct. If it's aligned with the given reference, then it is right

In [2]:
reference1_1 = np.load('./HumanML3D/new_joints/012314.npy')
reference2_1 = np.load('./HumanML3D/new_joint_vecs/012314.npy')

In [3]:
abs(reference1 - reference1_1).sum()

0.0

In [4]:
abs(reference2 - reference2_1).sum()

0.0